In [39]:
# imoport libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.decomposition import NMF
import pickle
from fuzzywuzzy import process
import random

In [6]:
# import the data set of movie ratings 
df = pd.read_csv('../data/ml-latest-small/ratings.csv', na_values = 'Nan')
# import the data set of movie names
df_movie = pd.read_csv('../data/ml-latest-small/movies.csv', na_values = 'Nan')

In [7]:
# Split the movie name from movie year and apply it in the matrix
df_movie['year'] = df_movie.title.astype(str).str[-6:]
df_movie['title_new'] = df_movie.title.astype(str).str[:-6]

In [124]:
# make a frame with the Genres
# Try to zip columns with movie names
genres_movies = pd.DataFrame(list(zip(df_movie.genres, df_movie.title_new)),columns=['genres','movies'])
    # make column index
genres_movies.set_index('movies', inplace=True)
# Split multiple genres in several columns
genres_movies= genres_movies['genres'].str.split('|')
genres_movies


movies
Toy Story                              [Adventure, Animation, Children, Comedy, Fantasy]
Jumanji                                                   [Adventure, Children, Fantasy]
Grumpier Old Men                                                       [Comedy, Romance]
Waiting to Exhale                                               [Comedy, Drama, Romance]
Father of the Bride Part II                                                     [Comedy]
                                                             ...                        
Black Butler: Book of the Atlantic                  [Action, Animation, Comedy, Fantasy]
No Game No Life: Zero                                       [Animation, Comedy, Fantasy]
Flint                                                                            [Drama]
Bungo Stray Dogs: Dead Apple                                         [Action, Animation]
Andrew Dice Clay: Dice Rules                                                    [Comedy]
Name: genres, 

In [4]:
# use pivot to make the matrix of movie rates
rates =df.pivot(index='userId',columns = 'movieId')
R = rates.rating

In [8]:
# Try to zip columns with movie names
#new_columns = dict(zip(df.movieId,df_movie.title))
R.rename(columns=dict(zip(df_movie["movieId"], df_movie["title_new"])),inplace = True)
R.head()

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


movieId,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,Gintama: The Movie,anohana: The Flower We Saw That Day - The Movie,Silver Spoon,Love Live! The School Idol Movie,Jon Stewart Has Left the Building,Black Butler: Book of the Atlantic,No Game No Life: Zero,Flint,Bungo Stray Dogs: Dead Apple,Andrew Dice Clay: Dice Rules
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# fill in the NaN values with zeros
# impute the nan with 0:
R = R.fillna(value=0)
R.sample(5)

movieId,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,Gintama: The Movie,anohana: The Flower We Saw That Day - The Movie,Silver Spoon,Love Live! The School Idol Movie,Jon Stewart Has Left the Building,Black Butler: Book of the Atlantic,No Game No Life: Zero,Flint,Bungo Stray Dogs: Dead Apple,Andrew Dice Clay: Dice Rules
userId,,,,,,,,,,,,,,,,,,,,,
379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
552,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68,2.5,2.5,2.0,0.0,2.0,4.0,2.0,0.0,0.0,4.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166,5.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,4.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Create the functions for the cosine similarity table
def cos_sim(vec1, vec2):
    """function to calcualte the cosine similarity between two vectors""" 
    num = np.dot(vec1, vec2)
    denom = np.sqrt(np.dot(vec1, vec1)) * np.sqrt(np.dot(vec2, vec2))
    return num / denom

def cos_sim_table(R):
    data = []
    for user1 in R.index:
        row = []
        for user2 in R.index:
            cs = cos_sim(R.loc[user1],R.loc[user2])
            row.append(cs)
        data.append(row)
    table = pd.DataFrame(data,index=R.index, columns=R.index)
    return table

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

In [46]:
R.T

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
Toy Story,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
Jumanji,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
Grumpier Old Men,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
Waiting to Exhale,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Father of the Bride Part II,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Black Butler: Book of the Atlantic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
No Game No Life: Zero,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Flint,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
# Returns numpy array:
cosine_similarity(R)

array([[1.        , 0.02728287, 0.05972026, ..., 0.29109737, 0.09357193,
        0.14532081],
       [0.02728287, 1.        , 0.        , ..., 0.04621095, 0.0275654 ,
        0.10242675],
       [0.05972026, 0.        , 1.        , ..., 0.02112846, 0.        ,
        0.03211875],
       ...,
       [0.29109737, 0.04621095, 0.02112846, ..., 1.        , 0.12199271,
        0.32205486],
       [0.09357193, 0.0275654 , 0.        , ..., 0.12199271, 1.        ,
        0.05322546],
       [0.14532081, 0.10242675, 0.03211875, ..., 0.32205486, 0.05322546,
        1.        ]])

In [43]:
# We can turn this into a dataframe:
cos_sim_table = pd.DataFrame(cosine_similarity(R),index=R.index, columns = R.index)
cos_sim_table

ValueError: Shape of passed values is (9724, 9724), indices imply (610, 610)

In [40]:
def create_new_user(R = R, k = 5):
    """
    In this function we initialize a new user according to a list of movies given 
    in the R Data-Frame. The unseen movies are considered as zero rating.
    This function is used for the cosine similarity suggesting function.
    k is the number of movies to ask for a rating
    
    """
    # initiate a new user
    new_user = np.zeros_like(R.columns)
    films = R.columns
    rand_movies = random.choices(films,k=k)
    user_votes_ran_movies = []
    for idx, movies in enumerate(rand_movies):
        user_votes_ran_movies.append(input(f'Give me the rating for the movie {movies}.'))
    # make a list out of the 
    new_user_ratings = pd.DataFrame(list(zip(rand_movies, user_votes_ran_movies)),columns=['movie','rating'])
    # make column index
    new_user_ratings.set_index('movie', inplace=True)
    for idx,item in enumerate(films):
        if new_user_ratings.get(item):   # returns the value for key if the key is in dictionary
            new_user[idx] = new_user_ratings[item]
    # new user dataframe
    new_user_rates = pd.DataFrame([new_user], index=['new_user'],columns = films)
    return new_user_rates

In [41]:
new_user_rates = create_new_user(R,5)
new_user_rates

movieId,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,Gintama: The Movie,anohana: The Flower We Saw That Day - The Movie,Silver Spoon,Love Live! The School Idol Movie,Jon Stewart Has Left the Building,Black Butler: Book of the Atlantic,No Game No Life: Zero,Flint,Bungo Stray Dogs: Dead Apple,Andrew Dice Clay: Dice Rules
new_user,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
